In [1]:
from google.colab import drive
import os
# Install p7zip-full for handling .7z archives
!apt-get install -y p7zip-full
!pip install nibabel nilearn dicom2nifti
from scipy.ndimage import shift

from nilearn.image import clean_img, resample_to_img
import nibabel as nib
import os
import dicom2nifti
import nibabel as nib
from nilearn.image import clean_img, resample_to_img, smooth_img, resample_img
from nilearn.masking import compute_brain_mask
from nilearn.datasets import load_mni152_template
import numpy as np
from nilearn.decomposition import CanICA
from sklearn.preprocessing import StandardScaler
from nilearn.connectome import ConnectivityMeasure
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Ridge
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import pandas as pd
import glob
!pip install scikit-image
from skimage.registration import phase_cross_correlation



Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
p7zip-full is already the newest version (16.02+dfsg-8).
0 upgraded, 0 newly installed, 0 to remove and 29 not upgraded.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 27.4 MB/s eta 0:00:00


In [2]:
# Mount Google Drive
drive.mount('/content/drive')

# Define paths
uploaded_file_path = '/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z.001'
output_dir = '/content/extracted_data'
os.makedirs(output_dir, exist_ok=True)

print("Google Drive mounted and paths defined.")


Mounted at /content/drive
Google Drive mounted and paths defined.


In [ ]:
# Rename the file to .7z
renamed_file_path = uploaded_file_path.replace('.7z.001', '.7z')
os.rename(uploaded_file_path, renamed_file_path)

print(f"File renamed to: {renamed_file_path}")


File renamed to: /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z


In [3]:
# Extract the renamed .7z file
!7z x "/content/drive/MyDrive/schizconnect_COBRE_images_22498.7z" -o"/content/extracted_data"


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.00GHz (50653),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan /content/drive/MyDrive/                                 1 file, 2277077170 bytes (2172 MiB)

Extracting archive: /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
--
Path = /content/drive/MyDrive/schizconnect_COBRE_images_22498.7z
Type = 7z
Physical Size = 2277077170
Headers Size = 13601
Method = LZMA2:24
Solid = +
Blocks = 2

  0%      1% 308 - COBRE/sub-A00000368/ses-20110 . 20110101_task-rest_bold.nii.gz                                                                           2% 328 - COBRE/sub-A00000541/ses-20100 . 20100

In [4]:
# List the extracted files
'''for root, dirs, files in os.walk(output_dir):
    for file in files:
        print(os.path.join(root, file))'''

save_dir = '/content/fmri/corrected_nifti_data/'
os.makedirs(save_dir, exist_ok=True)

process_dir = '/content/fmri/processed_imaging_data/'
os.makedirs(process_dir, exist_ok=True)

spatial_maps_dir = '/content/fmri/spatial_maps_data/'
os.makedirs(spatial_maps_dir, exist_ok=True)
ica = CanICA(n_components=20, mask_strategy='background')
scaler = StandardScaler()

In [5]:
import os
import glob
import nibabel as nib
import numpy as np
from nilearn.image import resample_img, resample_to_img, smooth_img, clean_img
from nilearn.masking import compute_brain_mask
from nilearn.input_data import NiftiMasker
from nilearn.datasets import load_mni152_template
from skimage.registration import phase_cross_correlation
from scipy.ndimage import shift
from sklearn.preprocessing import StandardScaler
from joblib import Parallel, delayed
from nilearn.maskers import NiftiMasker
from nilearn.decomposition import CanICA

# Initialize scaler globally for reuse
scaler = StandardScaler()

import os
import glob
import nibabel as nib
import numpy as np
from nilearn.image import resample_to_img, smooth_img, clean_img
from nilearn.datasets import load_mni152_template, fetch_atlas_harvard_oxford
from nilearn.maskers import NiftiLabelsMasker
from nilearn.input_data import NiftiMasker
from sklearn.preprocessing import StandardScaler

# Load Harvard-Oxford Atlas (ROIs)
ho_atlas = fetch_atlas_harvard_oxford('cort-maxprob-thr25-2mm')  # 96 cortical ROIs
ho_masker = NiftiLabelsMasker(labels_img=ho_atlas.maps, standardize=True, detrend=True,
                             low_pass=0.1, high_pass=0.01, t_r=2.0)

def extract_roi_features(nifti_img):
    """Extract mean ROI time-series features from fMRI data."""
    # Motion correction (use Nilearn's default instead of custom)
    # Note: If motion correction is already done, skip this step.
    cleaned_img = clean_img(nifti_img, detrend=True, standardize="zscore", t_r=2.0)

    # Smoothing
    smoothed_img = smooth_img(cleaned_img, fwhm=6)

    # Extract ROI time-series (shape: [time_points, n_rois])
    roi_time_series = ho_masker.fit_transform(smoothed_img)

    # Feature engineering: Temporal variability (mean/std across time)
    roi_means = np.mean(roi_time_series, axis=0)   # Mean activation per ROI
    roi_stds = np.std(roi_time_series, axis=0)     # Temporal variability per ROI

    # Combine into feature vector (shape: [n_samples, n_features])
    features = np.concatenate([roi_means, roi_stds])  # 96*2 = 192 features

    return features

# Main processing loop (simplified)
def process_nifti_files(nifti_files):
    features = []
    for i, nii_file in enumerate(nifti_files):
        print(f"Processing {i+1}/{len(nifti_files)}: {nii_file}")
        img = nib.load(nii_file)
        features.append(extract_roi_features(img))
    return np.array(features)

nifti_file_path = '/content/extracted_data/COBRE/sub*/ses*/func/*.nii.gz'

# Glob all files
nifti_files = glob.glob(nifti_file_path)
# Debugging step to print the files being processed
def is_empty_nifti(file_path):
    img = nib.load(file_path)
    return np.count_nonzero(img.get_fdata()) == 0  # Check if data is empty

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if not is_empty_nifti(f)]
def is_valid_nifti(file_path):
    try:
        nib.load(file_path)
        return True
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return False

nifti_files = glob.glob(nifti_file_path)
nifti_files = [f for f in nifti_files if is_valid_nifti(f)]

print("NIfTI files found:", nifti_files)

# Run the pipeline
X = process_nifti_files(nifti_files)

print("Preprocessing complete!")


[_add_readme_to_default_data_locations] Added README.md to /root/nilearn_data

[get_dataset_dir] Dataset created in /root/nilearn_data/fsl

[fetch_single_file] Downloading data from https://www.nitrc.org/frs/download.php/9902/HarvardOxford.tgz ...

[_chunk_report_] Downloaded 16367616 of 25716861 bytes (63.6%%,    0.6s remaining)

[fetch_single_file]  ...done. (2 seconds, 0 min)

[uncompress_file] Extracting data from /root/nilearn_data/fsl/5c734f16e50cc772ef593cab9bb3137b/HarvardOxford.tgz...

[uncompress_file] .. done.

NIfTI files found: ['/content/extracted_data/COBRE/sub-A00028402/ses-20120101/func/sub-A00028402_ses-20120101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00016720/ses-20110101/func/sub-A00016720_ses-20110101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00016720/ses-20100101/func/sub-A00016720_ses-20100101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00021598/ses-20110101/func/sub-A00021598_ses-20110101_task-rest_run-02_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00021598/ses-20110101/func/sub-A00021598_ses-20110101_task-rest_run-01_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00031186/ses-20120101/func/sub-A00031186_ses-20120101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00024510/ses-20090101/func/sub-A00024510_ses-20090101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00001181/ses-20100101/func/sub-A00001181_ses-20100101_task-rest_bold.nii.gz', '/content/extracted_data/COBRE/sub-A00018434/ses-20100

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans

# ... (Your previous code) ...

# Ensure all feature vectors are NumPy arrays and have the same length
#feature_vectors = [np.array(f) for f in feature_vectors]

# Find the maximum feature vector length
max_length = max(len(sublist) for f in feature_vectors for sublist in f)  # Modified to find max length across all sublists

# Pad all vectors to the same length
X = np.array([[np.pad(sublist, (0, max_length - len(sublist)), mode='constant')
              for sublist in f] for f in feature_vectors])  # Pad each sublist

print("Final feature matrix shape:", X.shape)  # Should be (num_samples, max_length)

# ... (Rest of your code) ...

Final feature matrix shape: (6, 50, 2196)


In [7]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.cluster import KMeans


# Define Autoencoder class
class Autoencoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim)
        )
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim)
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Initialize Autoencoder
print(X.shape, X.shape[0], X.shape[1])
input_dim = X.shape[1]  # Ensure correct shape after padding
latent_dim = 32
autoencoder = Autoencoder(input_dim, latent_dim)

# Set up optimizer and loss function
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
criterion = nn.MSELoss()

# Convert X to a PyTorch tensor
X_tensor = torch.FloatTensor(X)

# Train Autoencoder
num_epochs = 100
for epoch in range(num_epochs):
    autoencoder.train()
    optimizer.zero_grad()
    encoded, decoded = autoencoder(X_tensor)
    loss = criterion(decoded, X_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {loss.item():.4f}")

# Extract latent features
autoencoder.eval()
with torch.no_grad():
    fmri_features_old, _ = autoencoder(X_tensor)

# Convert features to NumPy for clustering
fmri_features_np = fmri_features_old.numpy()
print('dim', fmri_features_np.shape)


(102, 96) 102 96
Epoch [10/100], Loss: 0.3194
Epoch [20/100], Loss: 0.0621
Epoch [30/100], Loss: 0.0099
Epoch [40/100], Loss: 0.0049
Epoch [50/100], Loss: 0.0007
Epoch [60/100], Loss: 0.0006
Epoch [70/100], Loss: 0.0002
Epoch [80/100], Loss: 0.0001
Epoch [90/100], Loss: 0.0000
Epoch [100/100], Loss: 0.0000
dim (102, 32)


In [8]:
import zipfile
# Mount Google Drive
drive.mount('/content/drive')
# Define paths
uploaded_file_path = '/content/drive/MyDrive/dataverse_files.zip'
data_dir = '/content/extracted_eeg_data'
os.makedirs(data_dir, exist_ok=True)


# Extract the contents of the zip file
with zipfile.ZipFile(uploaded_file_path, 'r') as zip_ref:
    zip_ref.extractall(data_dir)
print("Google Drive mounted and paths defined.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted and paths defined.


In [9]:
!pip install mne
import os
import numpy as np
import mne
from mne.io import read_raw_edf
from mne.channels import make_standard_montage

# Constants
#DATA_DIR = "/path/to/edf/files"  # Update this
# Usage example
output_dir = "/content/eeg_features/"
os.makedirs(output_dir, exist_ok=True)

# Define electrode groups for brain regions (simplified)
REGIONS = {
    "frontal": ["Fp1", "Fp2", "F3", "F4", "F7", "F8"],
    "central": ["C3", "C4", "Cz"],
    "temporal": ["T3", "T4", "T5", "T6"],
    "parietal": ["P3", "P4", "Pz"],
    "occipital": ["O1", "O2"]
}

BANDS = {
    "delta": (0.5, 4),
    "theta": (4, 8),
    "alpha": (8, 12),
    "beta": (12, 30)
}

def extract_eeg_features(raw, label):
    """Extract region-specific spectral power (mean + std) per subject."""
    # Preprocess
    raw.pick_types(eeg=True)
    raw.set_montage(make_standard_montage("standard_1020"))  # Ensure electrode positions
    raw.filter(0.5, 30., fir_design='firwin')  # Skip gamma (noisy in EEG)

    # Extract epochs (non-overlapping 2s windows)
    epochs = mne.make_fixed_length_epochs(raw, duration=2, preload=True)

    # Compute PSD for all epochs
    psds, freqs = mne.time_frequency.psd_array_welch(
        epochs.get_data(),
        sfreq=raw.info["sfreq"],
        fmin=0.5,
        fmax=30,
        n_fft=256
    )

    # Aggregate features per region and band
    features = []
    for region, channels in REGIONS.items():
        # Find indices of channels in the region
        chan_indices = [i for i, ch in enumerate(raw.ch_names) if ch in channels]
        if not chan_indices:
            continue  # Skip if no channels found

        for band, (fmin, fmax) in BANDS.items():
            # Compute mean power across epochs and channels in the region
            band_mask = (freqs >= fmin) & (freqs < fmax)
            band_power = psds[:, chan_indices, :][..., band_mask].mean(axis=(0, 1, 2))
            band_std = psds[:, chan_indices, :][..., band_mask].std(axis=(0, 1, 2))

            features.extend([band_power, band_std])

    # Add label and save (one file per subject)
    features.append(label)
    output_file = os.path.join(output_dir, f"{os.path.splitext(file_name)[0]}_features.npy")
    np.save(output_file, np.array(features))

# Process files
for file_name in os.listdir(data_dir):
    if file_name.endswith(".edf"):
        file_path = os.path.join(data_dir, file_name)
        try:
            raw = read_raw_edf(file_path, preload=True)
            label = 0 if file_name.startswith("h") else 1  # Update label logic
            extract_eeg_features(raw, label)
        except Exception as e:
            print(f"Error processing {file_name}: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 67.2 MB/s eta 0:00:00
Extracting EDF parameters from /content/extracted_eeg_data/s14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 542499  =      0.000 ...  2169.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
1085 matching events found
No baseline correction ap

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.3s


0 bad epochs dropped
Effective window size : 1.024 (s)
Extracting EDF parameters from /content/extracted_eeg_data/h13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 241249  =      0.000 ...   964.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
482 matching events found
No baseline correction applied
0 projection item

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Extracting EDF parameters from /content/extracted_eeg_data/h12.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 224999  =      0.000 ...   899.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
450 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 450 event

...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.2s


Extracting EDF parameters from /content/extracted_eeg_data/h02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 227499  =      0.000 ...   909.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
455 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 455 event

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Extracting EDF parameters from /content/extracted_eeg_data/h05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 236249  =      0.000 ...   944.996 secs...
NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (6.604 s)

Not setting metadata
472 matching events found
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 472 event

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import numpy as np
import os
import numpy as np

def load_eeg_features(data_dir):
    """
    Load EEG features aggregated per subject
    Returns:
        eeg_features: np.array of shape [n_subjects, n_features]
        eeg_labels: np.array of shape [n_subjects]
    """
    subject_features = []
    subject_labels = []

    # Validate directory
    if not os.path.isdir(data_dir):
        raise ValueError(f"Invalid data directory: {data_dir}")

    # Load and process each file
    for file_name in sorted(os.listdir(data_dir)):
        if file_name.endswith("_features.npy"):
            try:
                file_path = os.path.join(data_dir, file_name)

                # Load data
                data = np.load(file_path)

                # Validate data shape
                if data.ndim != 1:
                    raise ValueError(f"Expected 1D array in {file_name}, got {data.ndim}D")

                # Separate features and labels
                features = data[:-1]  # All elements except last
                label = data[-1]      # Last element as label

                # Append to lists
                subject_features.append(features)
                subject_labels.append(label)

            except Exception as e:
                print(f"Skipping {file_name} due to error: {str(e)}")
                continue

    # Convert to numpy arrays
    eeg_features = np.array(subject_features)
    eeg_labels = np.array(subject_labels)

    # Final validation
    if len(eeg_features) == 0:
        raise ValueError("No valid EEG data found")

    if eeg_features.shape[0] != eeg_labels.shape[0]:
        raise ValueError("Feature/label count mismatch")

    print(f"Loaded {len(eeg_features)} subjects with {eeg_features.shape[1]} features")
    return eeg_features, eeg_labels


# Directory containing EEG feature files
output_dir = "/content/eeg_features/"
eeg_features, y = load_eeg_features(output_dir)

# Split into training and testing sets
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from torch.utils.data import DataLoader, TensorDataset

# Assuming X and y are already loaded correctly
# X: Features matrix (shape: 14411, 95)
# y: Labels vector (shape: 14411,)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(eeg_features, y, test_size=0.25, random_state=0)
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()
X_train = ss.fit_transform(X_train)  # Fit & transform on train
X_test = ss.transform(X_test)

def add_noise(X, noise_level=0.01):
    noise = noise_level * np.random.randn(*X.shape)
    return X + noise

X_train_noisy = add_noise(X_train, noise_level=0.02)
X_train_combined = np.vstack((X_train, X_train_noisy))
y_train_combined = np.hstack((y_train, y_train))  # Duplicate labels

# Convert to tensors again
X_train_combined = torch.tensor(X_train_combined, dtype=torch.float32)
y_train_combined = torch.tensor(y_train_combined, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)

# Create DataLoader for batching and shuffling
train_data = TensorDataset(X_train_combined, y_train_combined)
test_data = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)

# Define the model
class EEGModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(EEGModel, self).__init__()
        self.eeg_branch = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.5),  # Dropout for regularization
            nn.Linear(256, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)  # Output layer for classification (2 classes)
        )

    def forward(self, x):
        return self.eeg_branch(x)

# Initialize the model
model = EEGModel(input_dim=eeg_features.shape[1], hidden_dim=256, output_dim=2)  # Adjust hidden_dim as needed

# Define loss function and optimizer
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Update the loss function with class weights
criterion = nn.CrossEntropyLoss(weight=class_weights)

# Define the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop (use the same code you already have)
# Training loop (use the same code you already have)
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Add learning rate scheduler
scheduler = ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# Modify training loop
best_loss = float('inf')
for epoch in range(100):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss}")

    # Check for early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        print("No improvement, applying LR scheduler")
        scheduler.step(avg_loss)  # Reduce learning rate if loss stops improving


# Evaluate the model
import torch
import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score

# Ensure model is in evaluation mode
model.eval()

# Store predictions and ground truth labels
y_true = []
y_pred = []
y_probs = []  # For probability scores

# Disable gradient computation for evaluation
with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs)  # Get raw logits
        probs = torch.softmax(outputs, dim=1)[:, 1]  # Convert to probabilities (class 1)
        _, predicted = torch.max(outputs, 1)  # Get class with highest score

        y_true.extend(labels.numpy())  # Actual labels
        y_pred.extend(predicted.numpy())  # Predicted labels
        y_probs.extend(probs.numpy())  # Probability scores

# Convert lists to numpy arrays
y_true = np.array(y_true)
y_pred = np.array(y_pred)
y_probs = np.array(y_probs)

# Compute evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_probs)
TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
# Print results
print(f"✅ Accuracy: {accuracy:.4f}")
print("\n🔹 Classification Report:\n", classification_rep)
print("\n🔹 Confusion Matrix:\n", conf_matrix)
print(f"\n🔹 ROC-AUC Score: {roc_auc:.4f}")
accuracy =  (TP + TN) / (TP + FP + TN + FN)
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred)
print(f"🔹 F1 Score: {f1:.4f}")
print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))




Loaded 28 subjects with 40 features


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 1, Loss: 0.7429595589637756
Epoch 2, Loss: 0.6996660232543945
Epoch 3, Loss: 0.6445249319076538
Epoch 4, Loss: 0.6304408311843872
Epoch 5, Loss: 0.6318258047103882
No improvement, applying LR scheduler
Epoch 6, Loss: 0.6116353273391724
Epoch 7, Loss: 0.6070401072502136
Epoch 8, Loss: 0.6039507389068604
Epoch 9, Loss: 0.5865933895111084
Epoch 10, Loss: 0.5854142904281616
Epoch 11, Loss: 0.5680020451545715
Epoch 12, Loss: 0.5787932872772217
No improvement, applying LR scheduler
Epoch 13, Loss: 0.5554186105728149
Epoch 14, Loss: 0.5572178959846497
No improvement, applying LR scheduler
Epoch 15, Loss: 0.5437723398208618
Epoch 16, Loss: 0.5345656275749207
Epoch 17, Loss: 0.513449490070343
Epoch 18, Loss: 0.4993666112422943
Epoch 19, Loss: 0.5157062411308289
No improvement, applying LR scheduler
Epoch 20, Loss: 0.5079628229141235
No improvement, applying LR scheduler
Epoch 21, Loss: 0.4681643843650818
Epoch 22, Loss: 0.4786515533924103
No improvement, applying LR scheduler
Epoch 23, Lo

In [15]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_class_weight

# -------------------
# Data Preprocessing
# -------------------
import numpy as np
from sklearn.decomposition import PCA

# -------------------
# Data Preprocessing
# -------------------
def pad_or_truncate(data, target_size, pad_value=0):
    """Pads with zeros or truncates the dataset to match the required size."""
    current_size = data.shape[0]
    if current_size < target_size:
        pad_size = target_size - current_size
        pad_shape = (pad_size,) + data.shape[1:]  # Keep feature dimensions unchanged
        pad_values = np.full(pad_shape, pad_value)  # Pad with specified value
        return np.vstack((data, pad_values))  # Stack padded data
    return data[:target_size]  # Truncate if larger

# Reshape fMRI features (if needed)
fmri_features = X.reshape(X.shape[0], -1)  # Ensure 2D shape

# Align fMRI features to EEG features' sample size
num_samples = eeg_features.shape[0]  # Use EEG features' sample size as the target

# Pad/truncate fMRI features to match EEG features' sample size
fmri_aligned = pad_or_truncate(fmri_features, num_samples)  # Pad/truncate fMRI features

# Align feature dimensions using PCA
n_components_fmri = min(fmri_aligned.shape[0], fmri_aligned.shape[1])  # Will be 28
pca_fmri = PCA(n_components=n_components_fmri)
fmri_reduced = pca_fmri.fit_transform(fmri_aligned)
fmri_padded = np.pad(fmri_reduced, ((0, 0), (0, 40 - n_components_fmri)), mode='constant')


# Verify shapes
print("EEG features shape:", eeg_features.shape)  # (28, 40)
print("EEG labels shape:", y.shape)  # (28,)
print(y)
print("fMRI features shape:", fmri_padded.shape)  # (28, 40)
X_combined = np.hstack((eeg_features, fmri_padded))
# Standardize Data
scaler = StandardScaler()
X_combined = scaler.fit_transform(X_combined)

# -------------------
# Data Augmentation (Adding Noise)
# -------------------
def add_noise(data, noise_level=0.05):
    noise = np.random.normal(0, noise_level, data.shape)
    return data + noise

# Augment Data
X_noisy = add_noise(X_combined, noise_level=0.05)
X_combined_augmented = np.vstack((X_combined, X_noisy))
y_combined_augmented = np.hstack((y, y))

# Convert to PyTorch tensors
X_combined_augmented = torch.tensor(X_combined_augmented, dtype=torch.float32)
y_combined_augmented = torch.tensor(y_combined_augmented, dtype=torch.float32)  # Float for BCE loss

# Split into train/test
X_train, X_test, y_train, y_test = train_test_split(X_combined_augmented, y_combined_augmented, test_size=0.25, random_state=42)

# Convert to PyTorch tensors
X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
y_train, y_test = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)

# Create DataLoaders
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=64, shuffle=True)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=64, shuffle=False)

# -------------------
# Define Fusion Model
# -------------------
class FusionModel(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(FusionModel, self).__init__()
        '''self.conv = nn.Sequential(
            nn.Conv1d(in_channels=1, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
            nn.Conv1d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool1d(kernel_size=2),
        )

        conv_output_size = (input_dim // 4) * 128  # Adjust based on pooling layers
'''
        self.fc = nn.Sequential(
            nn.Linear(input_dim, 1024),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(1024, 1)  # 1 output for binary classification
        )

    def forward(self, x):
        #x = x.unsqueeze(1)  # (batch_size, 1, feature_dim) for Conv1D
        #x = self.conv(x)
        #x = x.view(x.shape[0], -1)  # Flatten
        x = self.fc(x)
        return x  # No sigmoid here, we use BCEWithLogitsLoss


# Initialize the model
model = FusionModel(input_dim=X_combined_augmented.shape[1], hidden_dim=1024)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weights = torch.tensor(class_weights, dtype=torch.float32)

# Define Loss Function (with class weights)
criterion = nn.BCEWithLogitsLoss(pos_weight=class_weights[1])

# Define Optimizer
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Learning Rate Scheduler
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=3, factor=0.5, verbose=True)

# -------------------
# Training Loop
# -------------------
best_loss = float('inf')
for epoch in range(50):
    model.train()
    running_loss = 0.0

    for batch in train_loader:
        inputs, labels = batch
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs).squeeze(1)  # Ensure (batch_size,) shape
        loss = criterion(outputs, labels)

        # Backward pass
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    avg_loss = running_loss / len(train_loader)
    print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")

    # Save best model
    # Check for early stopping
    if avg_loss < best_loss:
        best_loss = avg_loss
        torch.save(model.state_dict(), 'best_model.pth')  # Save best model
    else:
        print("No improvement, applying LR scheduler")
        scheduler.step(avg_loss)  # Reduce learning rate if loss stops improving

# -------------------
# Evaluation
# -------------------
model.eval()
y_true, y_pred, y_probs = [], [], []

with torch.no_grad():
    for batch in test_loader:
        inputs, labels = batch
        outputs = model(inputs).squeeze(1)
        probs = torch.sigmoid(outputs)  # Convert logits to probabilities
        predictions = (probs > 0.5).float()  # Threshold at 0.5

        y_true.extend(labels.numpy())
        y_pred.extend(predictions.numpy())
        y_probs.extend(probs.numpy())

y_true, y_pred, y_probs = np.array(y_true), np.array(y_pred), np.array(y_probs)

# Metrics
# Compute evaluation metrics
accuracy = accuracy_score(y_true, y_pred)
classification_rep = classification_report(y_true, y_pred)
conf_matrix = confusion_matrix(y_true, y_pred)
roc_auc = roc_auc_score(y_true, y_probs)
TN, FP, FN, TP = confusion_matrix(y_true, y_pred).ravel()

print('True Positive(TP)  = ', TP)
print('False Positive(FP) = ', FP)
print('True Negative(TN)  = ', TN)
print('False Negative(FN) = ', FN)
# Print results
print(f"✅ Accuracy: {accuracy:.4f}")
print("\n🔹 Classification Report:\n", classification_rep)
print("\n🔹 Confusion Matrix:\n", conf_matrix)
print(f"\n🔹 ROC-AUC Score: {roc_auc:.4f}")
accuracy =  (TP + TN) / (TP + FP + TN + FN)
from sklearn.metrics import f1_score

f1 = f1_score(y_true, y_pred)
print(f"🔹 F1 Score: {f1:.4f}")
print('Accuracy of the binary classifier = {:0.3f}'.format(accuracy))


EEG features shape: (28, 40)
EEG labels shape: (28,)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1.]
fMRI features shape: (28, 40)
Epoch 1, Loss: 0.6990
Epoch 2, Loss: 0.6179
Epoch 3, Loss: 0.5895
Epoch 4, Loss: 0.5337
Epoch 5, Loss: 0.5070
Epoch 6, Loss: 0.4631
Epoch 7, Loss: 0.4501
Epoch 8, Loss: 0.4229
Epoch 9, Loss: 0.3820
Epoch 10, Loss: 0.3575
Epoch 11, Loss: 0.3380
Epoch 12, Loss: 0.3048
Epoch 13, Loss: 0.2841
Epoch 14, Loss: 0.2565
Epoch 15, Loss: 0.2267
Epoch 16, Loss: 0.2109
Epoch 17, Loss: 0.1962
Epoch 18, Loss: 0.1849
Epoch 19, Loss: 0.1595


<ipython-input-15-59693a562e8e>:76: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train, X_test = torch.tensor(X_train, dtype=torch.float32), torch.tensor(X_test, dtype=torch.float32)
<ipython-input-15-59693a562e8e>:77: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train, y_test = torch.tensor(y_train, dtype=torch.float32), torch.tensor(y_test, dtype=torch.float32)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch 20, Loss: 0.1449
Epoch 21, Loss: 0.1397
Epoch 22, Loss: 0.1262
Epoch 23, Loss: 0.1103
Epoch 24, Loss: 0.0995
Epoch 25, Loss: 0.0893
Epoch 26, Loss: 0.0823
Epoch 27, Loss: 0.0690
Epoch 28, Loss: 0.0662
Epoch 29, Loss: 0.0555
Epoch 30, Loss: 0.0521
Epoch 31, Loss: 0.0482
Epoch 32, Loss: 0.0402
Epoch 33, Loss: 0.0374
Epoch 34, Loss: 0.0305
Epoch 35, Loss: 0.0295
Epoch 36, Loss: 0.0280
Epoch 37, Loss: 0.0252
Epoch 38, Loss: 0.0234
Epoch 39, Loss: 0.0201
Epoch 40, Loss: 0.0238
No improvement, applying LR scheduler
Epoch 41, Loss: 0.0209
No improvement, applying LR scheduler
Epoch 42, Loss: 0.0167
Epoch 43, Loss: 0.0157
Epoch 44, Loss: 0.0157
No improvement, applying LR scheduler
Epoch 45, Loss: 0.0146
Epoch 46, Loss: 0.0127
Epoch 47, Loss: 0.0114
Epoch 48, Loss: 0.0117
No improvement, applying LR scheduler
Epoch 49, Loss: 0.0111
Epoch 50, Loss: 0.0111
True Positive(TP)  =  3
False Positive(FP) =  2
True Negative(TN)  =  7
False Negative(FN) =  2
✅ Accuracy: 0.7143

🔹 Classification Re

In [17]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, f1_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.utils import resample, compute_class_weight

# -------------------
# Data Preprocessing
# -------------------
'''print(eeg_features.shape)
print(fmri_features.shape)
print(y.shape)

# Ensure equal sample sizes
num_eeg_samples = eeg_features.shape[0]
num_fmri_samples = fmri_features.shape[0]
num_samples = max(num_eeg_samples, num_fmri_samples)

# Pad or truncate data
def pad_or_truncate(data, target_size):
    current_size = data.shape[0]
    if current_size < target_size:
        pad_size = target_size - current_size
        pad_shape = (pad_size,) + data.shape[1:]
        pad_values = np.zeros(pad_shape)
        return np.vstack((data, pad_values))
    return data[:target_size]

# Apply padding/truncation
X_eeg = pad_or_truncate(eeg_features, num_samples)
X_fmri = pad_or_truncate(fmri_features, num_samples)
y_target = pad_or_truncate(y.reshape(-1, 1), num_samples).flatten()

# Reshape fMRI features if necessary
X_fmri = X_fmri.reshape(num_samples, -1)

# Combine EEG and fMRI features
X_combined = np.hstack((X_eeg, X_fmri))

# Standardize data
ss = StandardScaler()
X_combined = ss.fit_transform(X_combined)'''

# -------------------
# Bootstrapping for Better Generalization
# -------------------
X_resampled, y_resampled = resample(X_combined, y, replace=True, n_samples=num_samples, random_state=42)

# -------------------
# K-Fold Cross-Validation (Leave 1 Fold for Final Testing)
# -------------------
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

splits = list(kf.split(X_resampled, y_resampled))  # Store indices to manually hold 1 fold for final testing

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y), y=y)
class_weight_dict = {i: class_weights[i] for i in range(len(class_weights))}

# Store metrics
all_accuracies, all_f1_scores, all_roc_auc = [], [], []

# Leave the last fold for final testing
train_folds = splits[:-1]  # First 4 folds for training
test_fold = splits[-1]     # Last fold for testing

# Cross-validation on first 4 folds
for fold, (train_idx, val_idx) in enumerate(train_folds):
    print(f"\nTraining fold {fold+1}")

    # Train-test split
    X_train, X_val = X_resampled[train_idx], X_resampled[val_idx]
    y_train, y_val = y_resampled[train_idx], y_resampled[val_idx]

    # Initialize Logistic Regression model
    model = LogisticRegression(C=0.01, class_weight='balanced', max_iter=500, solver='lbfgs')

    # Train the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_val)
    y_probs = model.predict_proba(X_val)[:, 1]

    # Compute metrics
    accuracy = accuracy_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)
    try:
        roc_auc = roc_auc_score(y_val, y_probs)
    except ValueError:
        roc_auc = None

    # Store results
    all_accuracies.append(accuracy)
    all_f1_scores.append(f1)
    if roc_auc is not None:
        all_roc_auc.append(roc_auc)

    # Print metrics
    print(f"🔹 Accuracy: {accuracy:.4f}")
    print(f"🔹 F1 Score: {f1:.4f}")
    if roc_auc is not None:
        print(f"🔹 ROC-AUC Score: {roc_auc:.4f}")
    print("🔹 Classification Report:\n", classification_report(y_val, y_pred))
    print("🔹 Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

# -------------------
# Compute Average Metrics Across Training Folds
# -------------------
print(f"\n🔹 Average Accuracy: {np.mean(all_accuracies):.4f}")
print(f"🔹 Average F1 Score: {np.mean(all_f1_scores):.4f}")
if all_roc_auc:
    print(f"🔹 Average ROC-AUC: {np.mean(all_roc_auc):.4f}")

# -------------------
# Final Testing on Held-Out Fold
# -------------------
test_idx = test_fold[1]  # Get test set indices
X_test, y_test = X_resampled[test_idx], y_resampled[test_idx]

# Predict on test set
y_test_pred = model.predict(X_test)
y_test_probs = model.predict_proba(X_test)[:, 1]

# Compute final test metrics
test_accuracy = accuracy_score(y_test, y_test_pred)
test_f1 = f1_score(y_test, y_test_pred)
try:
    test_roc_auc = roc_auc_score(y_test, y_test_probs)
except ValueError:
    test_roc_auc = None

# Print final test metrics
print("\n🔹 Final Test Results on Held-Out Fold:")
print(f"🔹 Test Accuracy: {test_accuracy:.4f}")
print(f"🔹 Test F1 Score: {test_f1:.4f}")
if test_roc_auc is not None:
    print(f"🔹 Test ROC-AUC Score: {test_roc_auc:.4f}")
print("🔹 Final Test Classification Report:\n", classification_report(y_test, y_test_pred))
print("🔹 Final Test Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))



Training fold 1
🔹 Accuracy: 1.0000
🔹 F1 Score: 1.0000
🔹 ROC-AUC Score: 1.0000
🔹 Classification Report:
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00         3
         1.0       1.00      1.00      1.00         3

    accuracy                           1.00         6
   macro avg       1.00      1.00      1.00         6
weighted avg       1.00      1.00      1.00         6

🔹 Confusion Matrix:
 [[3 0]
 [0 3]]

Training fold 2
🔹 Accuracy: 0.8333
🔹 F1 Score: 0.8000
🔹 ROC-AUC Score: 0.7778
🔹 Classification Report:
               precision    recall  f1-score   support

         0.0       0.75      1.00      0.86         3
         1.0       1.00      0.67      0.80         3

    accuracy                           0.83         6
   macro avg       0.88      0.83      0.83         6
weighted avg       0.88      0.83      0.83         6

🔹 Confusion Matrix:
 [[3 0]
 [1 2]]

Training fold 3
🔹 Accuracy: 0.8333
🔹 F1 Score: 0.8000
🔹 ROC-AUC